# Review reading level with one LLM

In this notebook we will use an LLM (GPT-4o) to evaluate a text and see what the reading level is. 

The texts that we evaluate are the 'Verhaal Speciaal' stories. 

### Contents
0. Installs and imports
1. Load the story
2. Set prompt and settings
3. Evaluate the text
4. Results


## 0. Installs and imports

In [1]:
import glob
import json
import os
import config
import openai
openai.__version__

'1.54.5'

In [2]:
#Initialize the OpenAI client 

os.environ['OPENAI_API_KEY'] = config.OpenAI_key # replace with your API-key, but don't keep it in your source code :-)

client = openai.Client()

## 1. Load the story

In [3]:
#select the latest file
json_files = glob.glob('*.json')
#print(json_files)
#sorted_files = sorted(json_files, key=os.path.getctime)
#print(sorted_files)
json_files.sort(key=os.path.getmtime)
latest_file = json_files[-1]
print(latest_file)

V_S_2024-11-28_15:35.json


In [4]:
with open(latest_file, 'r') as file:
    data = json.load(file)
generated_text = data['text']
print(generated_text)

Titel: Een Verdwaalde Zomerdag

Hoofdstuk 1

Verteller | Tijdens een zonnige zomerse dag, twee vrienden, Eddy en Jan, besloten een wandeling te maken in het uitgestrekte bos achter hun huis. Ze waren bevriend sinds de kleuterschool en deelden een gezamenlijke passie voor avontuur.

{char1} | eddy | "Jan, je herinnert toch wel de weg terug, toch?" vroeg Eddy, zijn vriend aankijkend met een lichte ongerustheid in zijn ogen.

{char2} | jan | "Zeker wel, Eddy. We hebben de kaart en het kompas bij ons," antwoordde Jan, geruststellend. Hij zwaaide met een kaart in de ene hand en een kompas in de andere.

Verteller | Terwijl ze verder wandelden, werden de bomen om hen heen dichter en hun voetsporen op het modderige pad vager. De zon begon langzaam onder te gaan en het bos werd donker.

{char1} | eddy | "Jan, ben je het zeker dat we de juiste richting op gaan? Het lijkt hier op een totaal andere plek dan waar we eerder zijn geweest," zei Eddy, zijn stem trillend van angst.

{char2} | jan | "Ja

## 2. Set prompt & settings

In [5]:
system_prompt = '''Je bent een expert in het beoordelen van teksten voor kinderboeken. Je gaat een tekst beoordelen of deze aan een bepaald niveau voldoet.'''

In [6]:
#variables based on the reading level settings
level_one = 'Leesniveau 1: Tekst bestaat uit korte woorden die je precies zo schrijft zoals je ze uitspreekt. Voorbeelden hiervan zijn maan, bos, man, terug, verdwaald. Er mogen dus geen woorden voorkomen met bijvoorbeeld sch- en -ng en -nk, -b, -d(t), -ch(t), -ooi, -aai, -oei, -eeuw, -ieuw, -uw. De zinnen zijn zo kort mogelijk. Elke zin begint op een nieuwe regel. Er komen geen in hoofdletters voor.'

level_two = 'Leesniveau 2: Tekst bestaat uit één- en tweelettergrepige woorden. Woorden met -sch en -ng mogen voorkomen.  Woorden mogen voorkomen eindigend op -nk, -b, -d(t), -ch(t), -ooi, -aai, -oei, -eeuw, -ieuw, -uw. Een-. Ook mogen tweelettergrepige woorden met twee of drie medeklinkers na elkaar zoals staart, botst, sprong, bankje, knappe, winkel  Er mogen ook enkele makkelijke drielettergrepige woorden voorkomen, zoals sinterklaas. De zinnen zijn meestal kort. Soms mogen samengestelde zinnen voorkomen. Hoofdletters worden gebruikt.'

level_three = 'Leesniveau 3: Er mogen woorden gebruikt worden met 3 of meer lettergrepen. De zinnen mogen langer zijn. De zinnen kunnen bestaan uit een hoofdzin met een bijzin. Gebruik van leenwoorden, zoals bureau, horloge, chauffeur is toegestaan. Ook eenvoudige leesmoeilijkheden van leenwoorden komen voor: i en y uitgesproken als ie; c uitgesproken als k of als s. Zinnen hoeven niet te beginnen op een nieuwe regel. Samengestelde zinnen mogen voorkomen. Hoofdletters worden gebruikt.'

level_four = 'Leesniveau 4: Er zijn geen beperkingen in woorden en zinslengte. Lastig te lezen leenwoorden (gamen), onbekende woorden (ov-pas, ornament) en leestekens (ideeën, ruïne) komen meer voor. Woorden eindigend op -ele, -eaal, -ueel, -iaal of -ieel, woorden met een trema komen ook voor. Ook woorden beginnend met /ch/ uitgesproken als /sj/, eindigend op –ge, uitgesproken als /zje/, eindigend op –isch, woorden met klinkerreeks, leenwoorden met eau, é of è.'

leesniveaus = level_one + level_two +level_three + level_four
leesniveaus

'Leesniveau 1: Tekst bestaat uit korte woorden die je precies zo schrijft zoals je ze uitspreekt. Voorbeelden hiervan zijn maan, bos, man, terug, verdwaald. Er mogen dus geen woorden voorkomen met bijvoorbeeld sch- en -ng en -nk, -b, -d(t), -ch(t), -ooi, -aai, -oei, -eeuw, -ieuw, -uw. De zinnen zijn zo kort mogelijk. Elke zin begint op een nieuwe regel. Er komen geen in hoofdletters voor.Leesniveau 2: Tekst bestaat uit één- en tweelettergrepige woorden. Woorden met -sch en -ng mogen voorkomen.  Woorden mogen voorkomen eindigend op -nk, -b, -d(t), -ch(t), -ooi, -aai, -oei, -eeuw, -ieuw, -uw. Een-. Ook mogen tweelettergrepige woorden met twee of drie medeklinkers na elkaar zoals staart, botst, sprong, bankje, knappe, winkel  Er mogen ook enkele makkelijke drielettergrepige woorden voorkomen, zoals sinterklaas. De zinnen zijn meestal kort. Soms mogen samengestelde zinnen voorkomen. Hoofdletters worden gebruikt.Leesniveau 3: Er mogen woorden gebruikt worden met 3 of meer lettergrepen. De z

In [7]:
# TO DO prompt samenstellen met de tekst niveaus.

#prompt = system_prompt + leesniveaus + "Analyseer op welk leesniveau de volgende tekst is geschreven. Antwoord alleen het leesniveau. De woorden: Verteller, ENDOFACT, char1, char2, Hoofdstuk neem je NIET mee in je beoordeling."  + generated_text
prompt = system_prompt + leesniveaus + "Analyseer op welk leesniveau de volgende tekst is geschreven. Antwoord als eerste het leesniveua en beschrijf daarna je redenering. De woorden: Verteller, ENDOFACT, char1, char2, Hoofdstuk neem je NIET mee in je beoordeling."  + generated_text
prompt

'Je bent een expert in het beoordelen van teksten voor kinderboeken. Je gaat een tekst beoordelen of deze aan een bepaald niveau voldoet.Leesniveau 1: Tekst bestaat uit korte woorden die je precies zo schrijft zoals je ze uitspreekt. Voorbeelden hiervan zijn maan, bos, man, terug, verdwaald. Er mogen dus geen woorden voorkomen met bijvoorbeeld sch- en -ng en -nk, -b, -d(t), -ch(t), -ooi, -aai, -oei, -eeuw, -ieuw, -uw. De zinnen zijn zo kort mogelijk. Elke zin begint op een nieuwe regel. Er komen geen in hoofdletters voor.Leesniveau 2: Tekst bestaat uit één- en tweelettergrepige woorden. Woorden met -sch en -ng mogen voorkomen.  Woorden mogen voorkomen eindigend op -nk, -b, -d(t), -ch(t), -ooi, -aai, -oei, -eeuw, -ieuw, -uw. Een-. Ook mogen tweelettergrepige woorden met twee of drie medeklinkers na elkaar zoals staart, botst, sprong, bankje, knappe, winkel  Er mogen ook enkele makkelijke drielettergrepige woorden voorkomen, zoals sinterklaas. De zinnen zijn meestal kort. Soms mogen same

## 3. Evaluate the text

In [8]:
#calling the OpenAI function
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    #model="o1-preview",
    model="gpt-4o",
)
chat_completion

ChatCompletion(id='chatcmpl-AakfDS4Gvf7NW1FwDKeua8LNOswHy', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Leesniveau 3\n\nRedenering: De tekst bevat woorden met drie of meer lettergrepen, zoals "gestalte", "natuurlijke", "verbeeldingskracht", "nachtelijke", en "vindingrijkheid". Daarnaast worden samengestelde zinnen en bijzinnen gebruikt, zoals in "Terwijl ze verder wandelden, werden de bomen om hen heen dichter en hun voetsporen op het modderige pad vager." Er zijn ook leenwoorden opgenomen zoals "filosoof" en "cartograaf". De zinnen variëren in lengte en bevatten af en toe complexere structuren die passen bij het niveau van leesniveau 3.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1733322547, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_7f6be3efb0', usage=CompletionUsage(completion_tokens=137, prompt_tokens=1937, total_tokens=2074,

In [9]:
evaluation_by_llm =chat_completion.choices[0].message.content
print(f' {latest_file} = {evaluation_by_llm}')

 V_S_2024-11-28_15:35.json = Leesniveau 3

Redenering: De tekst bevat woorden met drie of meer lettergrepen, zoals "gestalte", "natuurlijke", "verbeeldingskracht", "nachtelijke", en "vindingrijkheid". Daarnaast worden samengestelde zinnen en bijzinnen gebruikt, zoals in "Terwijl ze verder wandelden, werden de bomen om hen heen dichter en hun voetsporen op het modderige pad vager." Er zijn ook leenwoorden opgenomen zoals "filosoof" en "cartograaf". De zinnen variëren in lengte en bevatten af en toe complexere structuren die passen bij het niveau van leesniveau 3.


## 4. Results

(Edit this table manually)

| JSON | Leesniveau gevraagd | Beoordeling door LLM (2x) | Beoordeling door mij
|----------|----------|----------|----------|
| V_S_2024-11-28_10:39.json | 1/1 | 2 + 2 | ? |
| V_S_2024-11-28_10:44.json | 1/1 | 1 + 1 | ? |
| V_S_2024-11-28_10:45.json | 1/1 | 2 + 2 | ? |
| V_S_2024-11-28_10:48.json | 1/1 | 3 + 3 | ? |
| V_S_2024-11-28_11:05.json | 1/1/1 | 1 + 1 | ? |
| V_S_2024-11-28_11:06.json | 1/1/1 | 2 + 2 | ? |
| V_S_2024-11-28_11:07.json | 1/1/1 | 2 + 2 | ? |
| V_S_2024-11-28_12:15.json | 1/1/1 | 2 + 2 | 1 |
| V_S_2024-11-28_12:33.json | 1/1/1 | 1 + 1 | 1 |
| V_S_2024-11-28_12:35.json | 1/1/1 | 1 + 1 | 1 | 
| V_S_2024-11-28_12:42.json | 1/1/1 | 1 + 1 | ? |
| V_S_2024-11-28_12:44.json | 1/1/1 | 1 + 1 | ? |
| V_S_2024-11-28_12:46.json | 1/1/1 | 1 + 1 | ? |
| V_S_2024-11-28_12:50.json | 1/1/1 | 1 + 1 | ? |
| V_S_2024-11-28_14:33.json | 2/2/2 | 2 + 2 | ? |
| V_S_2024-11-28_15:05.json | 2/2/2 | 2 + 2 | ? |
| V_S_2024-11-28_15:07.json | 2/2/2 | 3 + 3 | ? |
| V_S_2024-11-28_15:08.json | 2/2/2 | 2 + 2 | ? |
| V_S_2024-11-28_15:10.json | 2/2/2 | 3 + 3 | ? |
| V_S_2024-11-28_15:15.json | 3/3/3 | 3 + 4 | ? |
| V_S_2024-11-28_15:18.json | 3/3/3 | 3 + 3 | ? |
| V_S_2024-11-28_15:20.json | 3/3/3 | 3 + 3 | ? |
| V_S_2024-11-28_15:21.json | 3/3/3 | 3 + 4  | ? |
| V_S_2024-11-28_15:22.json | 3/3/3 | 3 + 3 | ? |
| V_S_2024-11-28_15:25.json | 4/4/4 | 4 + 4 | ? |
| V_S_2024-11-28_15:27.json | 4/4/4 | 4 + 4 | ? |
| V_S_2024-11-28_15:29.json | 4/4/4 | 3 + 3 | ? |
| V_S_2024-11-28_15:31.json | 4/4/4 | 3 + 3 | ? |
| V_S_2024-11-28_15:35.json | 4/4/4 | 4 + 3 | ? |